In [4]:
from mastodon import Mastodon
from pyspark.sql import SparkSession
from kafka import KafkaProducer
import json
import time
import logging
import re
from datetime import datetime
import pytz

# 1. Configuration du système de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 2. Configuration de l'accès à l'API Mastodon
logging.info("Configuration de l'accès à l'API Mastodon...")
mastodon = Mastodon(
    access_token='m6d7yGLo1ScqQqfKehJz0YgzuIowJf2UsCK3fRvNBUY',  # Remplacez par votre jeton d'accès
    api_base_url='https://mastodon.social'  # L'URL de votre instance Mastodon
)

# 3. Créer une session Spark
logging.info("Création de la session Spark...")
spark = SparkSession.builder \
    .appName("MastodonToKafkaStream") \
    .getOrCreate()

# 4. Configuration du producteur Kafka
logging.info("Configuration du producteur Kafka...")
producer = KafkaProducer(
    bootstrap_servers='kafka:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# 5. Fonction pour récupérer les posts depuis l'API Mastodon et les transformer
def fetch_mastodon_posts():
    logging.info("Récupération des posts depuis Mastodon...")
    # Récupérer les toots en temps réel (ici on récupère les 100 derniers toots)
    toots = mastodon.timeline_public(limit=100)

    for toot in toots:
        # Extraire les informations du toot
        toot_data = {
            'id': toot['id'],
            'username': toot['account']['username'],
            'display_name': toot['account']['display_name'],
            'content': toot['content'],
            'created_at': toot['created_at'].isoformat(),
            'favourites_count': toot['favourites_count'],
            'reblogs_count': toot['reblogs_count'],
            'replies_count': toot['replies_count']
        }

        # Affichage du message récupéré
        logging.info(f"Toot récupéré : {toot_data}")

        # Envoyer les données à Kafka
        producer.send('Mostodon_topic_stream', toot_data)

    logging.info(f"{len(toots)} toots envoyés à Kafka.")

# 6. Stream en continu depuis Mastodon vers Kafka
def stream_mastodon_to_kafka():
    logging.info("Démarrage du stream Mastodon vers Kafka...")
    while True:
        try:
            fetch_mastodon_posts()
            logging.info("Attente de 30 secondes avant la prochaine récupération...")
            time.sleep(30)  # Attendre 30 secondes avant de récupérer les nouveaux posts
        except Exception as e:
            logging.error(f"Erreur lors du stream des données : {e}")

# 7. Lancer le stream
stream_mastodon_to_kafka()

# 8. Fermer le producteur Kafka à la fin (nécessaire si vous arrêtez le programme)
producer.close()
logging.info("Producteur Kafka fermé.")


2024-10-02 08:55:17,801 - INFO - Configuration de l'accès à l'API Mastodon...
2024-10-02 08:55:18,317 - INFO - Création de la session Spark...
2024-10-02 08:55:18,356 - INFO - Configuration du producteur Kafka...
2024-10-02 08:55:18,379 - INFO - <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connecting> [IPv4 ('172.30.0.4', 9092)]>: connecting to kafka:9092 [('172.30.0.4', 9092) IPv4]
2024-10-02 08:55:18,383 - INFO - Probing node bootstrap-0 broker version
2024-10-02 08:55:18,393 - INFO - <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connecting> [IPv4 ('172.30.0.4', 9092)]>: Connection complete.
2024-10-02 08:55:18,510 - INFO - Broker version identified as 2.5.0
2024-10-02 08:55:18,513 - INFO - Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
2024-10-02 08:55:18,539 - INFO - <BrokerConnection node_id=1 host=kafka:9092 <connected> [IPv4 ('172.30.0.4', 9092)]>: Closing connection. 
2024-10-02 08:55:18,578 - INFO - Démarrage du stream

KeyboardInterrupt: 